<a href="https://colab.research.google.com/github/matchbou/kaggle-study-20190410/blob/master/Kaggle%E3%81%AETitanic%E3%81%AE%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB%E3%82%92XGBoost%E3%81%A7%E8%A7%A3%E3%81%8F%20-%20sambaiz-net.pdf-201904151520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.sambaiz.net/article/168/
# KaggleのTitanicのチュートリアルをXGBoostで解く

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"matchbou","key":"fb2093338e9cd630d62e0949908398dc"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
# kaggleデータ置き場とjupyernoteのカレントディレクトリフォルダを作成

#!mkdir ./input
#!mkdir ./notebook
!pwd
!ls -a

/content
.  ..  .config	kaggle.json  sample_data


In [9]:
#osライブラリ読込してinputに移動⇒kaggleデータもダウンロード　⇒ notebookフォルダに移動
import os
#os.chdir('input')

#https://www.kaggle.com/c/titanic/data
!kaggle competitions download -c titanic

#os.chdir('../notebook')


  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 24.1MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 29.2MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.65MB/s]


In [10]:
!pwd
!ls -a

/content
.   .config		   kaggle.json	test.csv
..  gender_submission.csv  sample_data	train.csv


In [11]:
#上記操作の結果以下のディレクトリ配置となる

# /content/notebook このipynbのカレントディレクトリ
# /content/input カグルからダウンロードしたcsvファイルの置き場所

!pwd
!ls -a ../input


/content
ls: cannot access '../input': No such file or directory


In [0]:
#ここからカーネルの内容に踏み込む

In [0]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [0]:
def preprocess(df):
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Embarked'] = df['Embarked'].fillna('Unknown')
    df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
    df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2, 'Unknown': 3} ).astype(int)
    df = df.drop(['Cabin','Name','PassengerId','Ticket'],axis=1)
    return df

def train(df):
    train_x = df.drop('Survived', axis=1)
    train_y = df.Survived
    (train_x, test_x ,train_y, test_y) = train_test_split(train_x, train_y, test_size = 0.6, random_state = 42)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    param = {'max_depth':3, 'learning_rate': 0.6, 'objective':'binary:logistic' }
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(xgb.DMatrix(test_x))
    print(accuracy_score(preds.round(), test_y))

    return bst

def predict(bst, df):
    return bst.predict(xgb.DMatrix(df))

In [0]:
df = pd.read_csv('./train.csv')
df_test_origin = pd.read_csv('./test.csv')

In [18]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [19]:
df_test_origin.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [0]:
df = preprocess(df)
df_test = preprocess(df_test_origin)

In [21]:
bst = train(df)


0.8205607476635514


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
answer = predict(bst, df_test).round().astype(int)

In [0]:
submit_data =  pd.Series(answer, name='Survived', index=df_test_origin['PassengerId'])
submit_data.to_csv('submit2.csv', header=True)

In [0]:
from google.colab import files


files.download('submit2.csv')